In [13]:
!apt install gdal-bin -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
gdal-bin is already the newest version (2.2.3+dfsg-2).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.


In [27]:
!pip install -U pip
!pip install rasterio turfpy

     |████████████████████████████████| 1.7 MB 6.3 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 20.2.4
    Uninstalling pip-20.2.4:
      Successfully uninstalled pip-20.2.4
     |████████████████████████████████| 14.8 MB 5.7 MB/s            
  Attempting uninstall: numpy
    Found existing installation: numpy 1.11.3
    Uninstalling numpy-1.11.3:
      Successfully uninstalled numpy-1.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.6.0 requires requests<3,>=2.21.0, but you have requests 2.5.2 which is incompatible.
matplotlib 3.3.4 requires pillow>=6.2.0, but you have pillow 2.6.1 which is incompatible.


In [43]:
CRS = "EPSG:2056"
!cd test-data/sample && rm -f sample.vrt && gdalbuildvrt -a_srs $CRS sample.vrt *.tif

0...10..Warning 6: gdalbuildvrt does not support ungeoreferenced image. Skipping 20190914_1211_12504_0_4.tif
.20...30...40...50...60...70...80...90...100 - done.


In [44]:
INPUT_DATA = 'test-data/sample/sample.vrt'

In [67]:
DEST_FOLDER = 'test-data/output/'
!mkdir -p $DEST_FOLDER

In [55]:
import json
from turfpy.measurement import bbox_polygon, bbox

bbox_wgs84 = [9.30140,47.15686 , 9.31869,47.16590]
bbox_espg2056 = [2739329,1223884, 2744413,1226542]
polygon = bbox_polygon(bbox_wgs84)
polygon.geometry

{"coordinates": [[[9.3014, 47.15686], [9.31869, 47.15686], [9.31869, 47.1659], [9.3014, 47.1659], [9.3014, 47.15686]]], "type": "Polygon"}

In [69]:
import rasterio
import rasterio.mask
import rasterio.vrt

def cut_image(src_file, bounding_boxes, destination_folder):
    with rasterio.open(INPUT_DATA) as src:
        with rasterio.vrt.WarpedVRT(src, src_crs="EPSG:2056", crs='EPSG:4236') as vrt:
            for index, bounding_box in enumerate(bounding_boxes):
                polygon = bbox_polygon(bounding_box)
                out_image, out_transform = rasterio.mask.mask(vrt, [polygon.geometry], crop=True)
                out_meta = src.meta
                out_meta.update({
                    "driver": "GTiff",
                     "height": out_image.shape[1],
                     "width": out_image.shape[2],
                     "transform": out_transform,
                })
                with rasterio.open(f"{destination_folder}{index}.tiff", "w", **out_meta) as dest:
                    dest.write(out_image)

In [71]:
bbox_0 = [9.30140, 47.15686 , 9.31869, 47.16590]
bbox_1 = [9.294078,47.133949 , 9.311906,47.143270]

cut_image(INPUT_DATA, [bbox_0, bbox_1], DEST_FOLDER)


In [2]:
import matplotlib as plt

print('array type: ',type(thumbnail))
print(thumbnail)

plt.imshow(thumbnail)
plt.colorbar()
plt.title('Overview - Band 4 {}'.format(thumbnail.shape))
plt.xlabel('Column #')
plt.ylabel('Row #')

NameError: name 'thumbnail' is not defined